In [2]:
!pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11434 sha256=94a8bfec2ad0e3a2cfbb9baa06c0c37b1fcf579f57764e3a108932d39b9c696b
  Stored in directory: c:\users\angela\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying


In [4]:
!pip install chart_studio

In [5]:
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
orders = pd.read_csv('./Data/orders_export.csv')

C:\Users\Angela\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning:

Columns (31,41,44,69,70) have mixed types.Specify dtype option on import or set low_memory=False.



In [8]:
orders['Financial Status'].unique()

array(['paid', nan, 'partially_refunded', 'refunded', 'pending'],
      dtype=object)

In [9]:
orders_1 = pd.read_csv('./Data/orders_export_1.csv')
orders_2 = pd.read_csv('./Data/orders_export_2.csv')

In [10]:
orders_com = orders_1.append(orders_2, ignore_index=True)

In [11]:
orders_com['Financial Status'].unique()

array(['paid', nan, 'refunded', 'partially_refunded', 'pending', 'voided'],
      dtype=object)

In [12]:
orders['Order_Date'] = pd.to_datetime(orders['Created at'], format="%Y-%m-%d %H:%M:%S %z", utc=True)
orders['Date'] = pd.to_datetime(orders['Order_Date'].dt.date)
orders['Date_YM'] = orders['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [13]:
orders_com['Order_Date'] = pd.to_datetime(orders_com['Created at'], format="%Y-%m-%d %H:%M:%S %z", utc=True)
orders_com['Date'] = pd.to_datetime(orders_com['Order_Date'].dt.date)
orders_com['Date_YM'] = orders_com['Date'].apply(lambda x: x.strftime('%Y-%m'))

In [14]:
summary_de = orders.groupby(['Date_YM', 'Billing Country']).agg({'Subtotal':pd.Series.sum, 'Shipping':pd.Series.sum, 'Taxes':pd.Series.sum, 'Total':pd.Series.sum, 'Discount Amount':pd.Series.sum, 'Refunded Amount':pd.Series.sum}).reset_index()
summary_com = orders_com.groupby(['Date_YM', 'Billing Country']).agg({'Subtotal':pd.Series.sum, 'Shipping':pd.Series.sum, 'Taxes':pd.Series.sum, 'Total':pd.Series.sum, 'Discount Amount':pd.Series.sum, 'Refunded Amount':pd.Series.sum}).reset_index()

In [15]:
summary_de['TO_EXSHIP_EXVAT'] = summary_de['Subtotal'] - summary_de['Taxes']
summary_com['TO_EXSHIP_EXVAT'] = summary_com['Subtotal'] - summary_com['Taxes']

In [16]:
summary_de['TO_EXSHIP_EXVAT_EXRET'] = summary_de['TO_EXSHIP_EXVAT'] - summary_de['Refunded Amount']
summary_com['TO_EXSHIP_EXVAT_EXRET'] = summary_com['TO_EXSHIP_EXVAT'] - summary_com['Refunded Amount']

In [17]:
summary_de.to_clipboard(decimal=',', index=False)

In [18]:
summary_com.to_clipboard(decimal=',', index=False)

In [21]:
summary_de.head(2)

,Date_YM,Billing Country,Subtotal,Shipping,Taxes,Total,Discount Amount,Refunded Amount,TO_EXSHIP_EXVAT,TO_EXSHIP_EXVAT_EXRET
0,2019-12,AT,154.90,9.0,0.0,163.90,0.00,29.98,154.90,124.92
1,2019-12,DE,18159.65,203.5,470.3,18363.15,111.98,8536.57,17689.35,9152.78


In [22]:
summary_com.head(2)

,Date_YM,Billing Country,Subtotal,Shipping,Taxes,Total,Discount Amount,Refunded Amount,TO_EXSHIP_EXVAT,TO_EXSHIP_EXVAT_EXRET
0,2018-08,NL,1874.12,3.95,325.26,1878.07,0.00,0.0,1548.86,1548.86
1,2018-09,NL,683.25,19.75,118.59,699.05,252.37,0.0,564.66,564.66


### Find the return rate of 35% at brand level

In [90]:
refund_total = summary_de[summary_de['Date_YM']>= '2020-01'][summary_de['Date_YM'] < '2021-01']['Refunded Amount'].sum() 
TO = summary_de[summary_de['Date_YM']>= '2020-01'][summary_de['Date_YM'] < '2021-01']['TO_EXSHIP_EXVAT'].sum()
refund_total / TO 

<ipython-input-90-8869850c4b74>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-90-8869850c4b74>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.5010962064970836

In [110]:
refund_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Billing Country'].isin(['DE', 'AT', 'CH'])][orders['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX = orders[orders['Date'] >= '2020-01-01'][orders['Date']<'2021-01-01'][orders['Billing Country'].isin(['DE', 'AT', 'CH'])][orders['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()


<ipython-input-110-ef728d4ebebb>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-110-ef728d4ebebb>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [111]:
refund_TX / all_TX

0.415167865707434

In [103]:
refund_TX / all_TX

0.41438763376932225

In [104]:
refund_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()
refund_TX_com / all_TX_com

<ipython-input-104-0417e2c09336>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-104-0417e2c09336>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.3614678899082569

In [108]:
refund_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name'].nunique()
all_TX_com = orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['paid', 'partially_refunded', 'refunded'])]['Name'].nunique()
refund_TX_com / all_TX_com

<ipython-input-108-f47bfe02ab8c>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-108-f47bfe02ab8c>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.34392265193370164

Apparently on the .com site, the percentage of refund transactions is 36.14%

In [105]:
refund_total_com = summary_com[summary_com['Date_YM']>= '2020-01'][summary_com['Date_YM'] < '2021-01']['Refunded Amount'].sum() 
TO_com = summary_com[summary_com['Date_YM']>= '2020-01'][summary_com['Date_YM'] < '2021-01']['TO_EXSHIP_EXVAT'].sum()
refund_total_com / TO_com

<ipython-input-105-6c7fce056eab>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

<ipython-input-105-6c7fce056eab>:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



0.42170861956583394

In [109]:
orders_com[orders_com['Date'] >= '2020-01-01'][orders_com['Date']<'2021-01-01'][orders_com['Billing Country'].isin(['DE', 'AT', 'CH'])][orders_com['Financial Status'].isin(['partially_refunded', 'refunded'])]['Name']

<ipython-input-109-b577a625be2b>:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



3812     100020700
3838     100020682
3858     100020668
3862     100020664
3969     100020589
           ...    
21454    100011593
21470    100011585
21503    100011571
21520    100011563
21526    100011560
Name: Name, Length: 498, dtype: object